<a href="https://colab.research.google.com/github/ranjanchoubey/ml2/blob/main/assignment-1/Q1-Autoencoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<!-- ![image.png](attachment:image.png) -->

![heading.png](https://github.com/ranjanchoubey/ml2/blob/main/assignment-1/assets/heading.png?raw=1)

![q1.1.png](https://github.com/ranjanchoubey/ml2/blob/main/assignment-1/assets/q1.1.png?raw=1)


**Ans : -** **Denoising Autoencoder** (DAE) is a type of neural network that helps to find useful patterns in data.
- It is often used to reduce the number of features or to learn important features.

-  The key idea behind a Denoising Autoencoder is to train the network to reconstruct the original input from a corrupted version of it.

- This process forces the network to learn robust features that can capture the underlying structure of the data, even in the presence of noise.


**Pseudocode for Denoising Autoencoder Training Loop**

---

- **Step 1.** Initialize the autoencoder model $f_{\theta}$
- **Step 2.** Define the loss function $\mathcal{L}$ (e.g., Mean Squared Error)
- **Step 3.** Define the optimizer (e.g., Adam)
- **Step 4.** Set the number of epochs $N$ and batch size $B$
- **Step 5.** For each epoch $n$ from 1 to $N$:
    - For each batch of data $\{x_i\}_{i=1}^{B}$:
        - **i.** Add noise to the input data to create a corrupted version $\tilde{x} = x + \text{noise}$  
        - **ii.** Forward pass: Pass the corrupted data through the autoencoder to get the reconstructed output $\hat{x} = f_{\theta}(\tilde{x})$  
        - **iii.** Compute the loss $\mathcal{L}(\hat{x}, x)$ between the reconstructed output and the original input  
        - **iv.** Backward pass: Compute the gradients of the loss with respect to the model parameters $\nabla_{\theta} \mathcal{L}$  
        - **v.** Update the model parameters using the optimizer $\theta \leftarrow \theta - \eta \nabla_{\theta} \mathcal{L}$       
- **Step 6.** Evaluate the model on a validation set (optional)
- **Step 7.** Save the trained model (optional)

---


![q1.2.png](https://github.com/ranjanchoubey/ml2/blob/main/assignment-1/assets/q1.2.png?raw=1)